In [89]:
### incorporating multiple tables
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits, load_iris
from sklearn.metrics import accuracy_score
import time

class lsh:
    def __init__(self, hash_size, data_dim, num_tables,random_type=None):
        self.num_rand_vec = hash_size  # number of buckets will be 2**hash_size eg: 2**2=4 (00,01,10,11)
        self.dim = data_dim
        self.num_tables = num_tables
        self.hash_tables = [{} for _ in range(self.num_tables)]
        self.seeds = [i for i in range(self.num_tables)]
        self.random_vectors = []
        for seed in self.seeds:
            np.random.seed(seed)
            if random_type:
                self.random_vectors.append(self.gen_random_vectors(random_type='normal_gpu'))
            else:  # normal distribution by default
                self.random_vectors.append(self.gen_random_vectors())
        print('TENSORFLOW GPU $$$$$$$$$$$$$ available GPU is -->>,',tf.test.gpu_device_name())

    def gen_random_vectors(self,random_type=None):
        if random_type is None:
            # sample from random_normal distribution by default
            return np.random.randn(self.num_rand_vec, self.dim)
        if random_type == 'normal_gpu':
            return tf.random.normal((self.num_rand_vec, self.dim)).numpy()
        if random_type == 'uniform':
            return np.random.rand(self.num_rand_vec, self.dim)

#     @tf.function
    def make_hash_key(self, inp):
        return ''.join(inp)

    def fit(self,data,label):
        assert data.shape[1] == self.dim, 'dimension of input data is {} and dimension in LSH object is {}'.format(
            data.shape, self.dim)
        # sess = tf.Session()
        # sess = tf.compat.v1.InteractiveSession()
        print('fitting')
        gpu_availability = tf.test.is_gpu_available()
        print('is GPU availabale ->',gpu_availability)
#         if gpu_availability:
        session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
#         with tf.device("/GPU:0"):
        tf.compat.v1.disable_eager_execution()
#         print('creating session on gpu, eager execution disabled ')
        data_ph = tf.compat.v1.placeholder("float", [None, self.dim])
        rand_hash_vec_ph = tf.compat.v1.placeholder("float", [None, self.dim])
        distance_matrix_ = tf.matmul(data_ph, tf.transpose(rand_hash_vec_ph))
        euclidean_dist_ = tf.sqrt(tf.reduce_sum(distance_matrix_ ** 2, axis=1))
        # data_ph = tf.convert_to_tensor(data,dtype=tf.float32)
        distance_as_keys = tf.strings.as_string(tf.cast((distance_matrix_ > 0),tf.int8))
        keys_ = tf.strings.reduce_join(distance_as_keys,axis=1) # [b'1001101',b'11001011',....]
        unique_keys_,idx = tf.unique(keys_)
        sorted_distance_idx = tf.argsort(euclidean_dist_)
#         (distance, bucket_key, data)
        
        
#         tf.range(0,self.num_tables)
        for rand_vec, hash_table in zip(self.random_vectors, self.hash_tables):
            # rand_hash_vec_ph = rand_vec
            # distance_matrix_ = tf.matmul(data_ph, tf.transpose(rand_hash_vec_ph))
            # euclidean_dist_ = tf.sqrt(tf.reduce_sum(distance_matrix_ ** 2, axis=1))
            t1 = time.time()
            distance_matrix,euclidean_dist,keys, unique_keys, sorted_dist_idx = session.run([distance_matrix_, euclidean_dist_,keys_,
                                                                            unique_keys_,sorted_distance_idx],
                                                      feed_dict={data_ph: data,
                                                                 rand_hash_vec_ph: rand_vec})
            print(type(distance_matrix),'type(distance_matrix)',distance_matrix.shape)
            print('total keys',len(keys))
            print('unique_keys', len(unique_keys))
            print('time taken for matmul ', time.time() - t1)
            print(sorted_dist_idx,len(sorted_dist_idx))
            # distance_matrix, euclidean_dist = np.array(distance_matrix_), np.array(euclidean_dist_)
#             keys = list(map(self.make_hash_key, (distance_matrix > 0).astype('int').astype('str')))
            # print('euclidean dist', (distance_matrix > 0).astype('int'))

            # the keys contain string of length=hash_size (2 bits or 3 bits..) for each document.
            # Eg '00','01','10','11'  for hash_size of 2 bits.
#             unique_keys = set(keys)
            for key in unique_keys:
                hash_table[key] = []
            
            assert len(keys) == len(euclidean_dist), 'shape of euclidean dist matrix is {} and length of keys list'\
                                                     ' is {}. They must match'.format(len(euclidean_dist), len(keys))
#             sorted_keys = [(dist,key,key_idx) for dist, key, key_idx in
#                            sorted(zip(list(euclidean_dist),keys,range(len(keys))),  key=lambda pair: pair[0])]
            # key_idx represents the document index in original data. We need to preserve this info before sorting
            # the points in one bucket based on their distances from randomly projected vectors.
            for idx in sorted_dist_idx:
#                 print(keys[idx],euclidean_dist[idx], data[idx])
                hash_table[keys[idx]].append((euclidean_dist[idx], data[idx],label[idx]))
        
#             for key in sorted_keys:  # (dist,key,key_idx)
#                 hash_table[key[1]].append((key[0],key[2]))  # (distance, doc_idx) <<<<<<<< very important
                # each key:value pair in hash table looks like this
                # '01':[(euclidean_dist, document_idx_in_data), () ,() , ......]
                # '01' is a bucket, in which a document might be present in.
#         else:
#             print('fitting skipped')
#             print(hash_table.keys())
        return 'success'

    def sort_buckets_elements(self):
        """
        call this after function after dividing the data into several buckets.
        Idea is to sort the bucket elements based on distance from random projection vector. Given a query point,
        we can find the top k similar elements by doing exhaustive search in the bucket. But if the bucket elements
        are sorted we can find top k similar items without having to compare all the
        elements in a particular bucket. This will speedup query.
        """
        pass
    
    
    def hash_table(self):
        return self.hash_tables

    def hash_table_dist(self):
        distributions = []
        for hash_table in self.hash_tables:
            summary_of_table = {}
            for k, v in hash_table.items():
                summary_of_table[k] = len(v)
            distributions.append(summary_of_table)
        return distributions

    def query(self, query_data):
        key_for_each_table = []
        for rand_vec in self.random_vectors:
            key = ''.join((np.dot(query_data, rand_vec.T) > 0).astype('int').astype('str'))
            key_for_each_table.append(key)  # each point will be assigned to exactly on bucket in one hash table
        result = []
        assert len(key_for_each_table) == len(self.hash_tables), 'Some data point is not assigned to any bucket in a hash table'
        for hash_table, key in zip(self.hash_tables, key_for_each_table):
            if key in hash_table.keys():
                result.extend(hash_table[key])
        #         print(keys_for_each_table)
        return list(set(result))
    
    def fast_query(self, query_data):
        if len(query_data.shape) == 1:
            query_data = query_data.reshape((1,-1))
        euc_dist_with_key_for_each_table = []
        for rand_vec in self.random_vectors:
            distance_matrix = np.dot(query_data, rand_vec.T)
            euclidean_dist = np.sqrt(np.sum(distance_matrix ** 2, axis=1))
            key = list(map(self.make_hash_key, (distance_matrix > 0).astype('int').astype('str')))
#             print(key[0].encode('utf-8'),'key<<')
            euc_dist_with_key_for_each_table.append((euclidean_dist[0],key[0].encode('utf-8')))
            # each point will be assigned to exactly on bucket in one hash table.
            # euc_dist_with_key_for_each_table is a list of tuple-->
            # [(key in hash table, euclidean distance from rand vector), (), ...]
        result = []
        assert len(euc_dist_with_key_for_each_table) == len(self.hash_tables), 'Some data point is not assigned to any bucket in a hash table'
        for hash_table, distance_key_tuple, in zip(self.hash_tables, euc_dist_with_key_for_each_table):
            # print('<<<<<<<<<<<',euc_dist_with_key_for_each_table)
            distance = distance_key_tuple[0]
            # print('query_distance', distance)
            key = distance_key_tuple[1]
            if key in hash_table.keys():
                # now use the key( or bucket) of each hash table, along with euclidean distance from random projection
                # vectors to find the most similar items to the query doc, instead of exhaustive search.
                # Using binary search on distances.
                bucket_elements = hash_table[key]
                # print('bucket_elements', bucket_elements)
                candidates = self.binary_search(arr=bucket_elements,query_distance=distance,max_k=5)
                # print('candidates',candidates)
                result.extend(candidates)
        result_docs,labels = np.vstack([tupl[1] for tupl in result]), [tupl[2] for tupl in result]
#         print(result_docs,labels,',,,,,,,')
        return result_docs,labels

#     def fast_query(self, query_data):
#         if len(query_data.shape) == 1:
#             query_data = query_data.reshape((1,-1))
#         euc_dist_with_key_for_each_table = []
#         session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
#         tf.compat.v1.disable_eager_execution()
#         data_ph = tf.compat.v1.placeholder("float", [None, self.dim])
#         rand_hash_vec_ph = tf.compat.v1.placeholder("float", [None, self.dim])
#         distance_matrix_ = tf.matmul(data_ph, tf.transpose(rand_hash_vec_ph))
#         euclidean_dist_ = tf.sqrt(tf.reduce_sum(distance_matrix_ ** 2, axis=1))
#         # data_ph = tf.convert_to_tensor(data,dtype=tf.float32)
#         distance_as_keys = tf.strings.as_string(tf.cast((distance_matrix_ > 0),tf.int8))
#         keys_ = tf.strings.reduce_join(distance_as_keys,axis=1) # [b'1001101',b'11001011',....]
# #         tf.while_loop(i>0)
#         for rand_vec in self.random_vectors:
#             distance_matrix,euclidean_dist,keys = session.run([distance_matrix_, euclidean_dist_,keys_],
#                                                       feed_dict={data_ph: query_data,
#                                                                  rand_hash_vec_ph: rand_vec})
# #             distance_matrix = np.dot(query_data, rand_vec.T)
# #             euclidean_dist = np.sqrt(np.sum(distance_matrix ** 2, axis=1))
# #             key = list(map(self.make_hash_key, (distance_matrix > 0).astype('int').astype('str')))
#             print(type(distance_matrix),'type(distance_matrix)',distance_matrix.shape)
#             print('total keys',len(keys))
#             euc_dist_with_key_for_each_table.append((euclidean_dist,keys))
#             # each point will be assigned to exactly on bucket in one hash table.
#             # euc_dist_with_key_for_each_table is a list of tuple-->
#             # [(key in hash table, euclidean distance from rand vector), (), ...]
# #         print(euc_dist_with_key_for_each_table,'euc_dist_with_key_for_each_table', len(euc_dist_with_key_for_each_table))
# #         print(len(euc_dist_with_key_for_each_table[0]))
#         result = []
#         assert len(euc_dist_with_key_for_each_table) == len(self.hash_tables), 'Some data point is not assigned to any bucket in a hash table'
#         for hash_table, distance_key_tuple, in zip(self.hash_tables, euc_dist_with_key_for_each_table):
#             # print('<<<<<<<<<<<',euc_dist_with_key_for_each_table)
            
#             for dist_key in distance_key_tuple:
#                 print('dist_key', dist_key)
#                 key = dist_key[1]
#                 distance = dist_key[0]
#                 print('key and bucket', key, distance)
#                 if key in hash_table.keys():
#                     # now use the key( or bucket) of each hash table, along with euclidean distance from random projection
#                     # vectors to find the most similar items to the query doc, instead of exhaustive search.
#                     # Using binary search on distances.
#                     bucket_elements = hash_table[key]
#                     # print('bucket_elements', bucket_elements)
#                     candidates = self.binary_search(arr=bucket_elements,query_distance=distance,max_k=5)
#                     # print('candidates',candidates)
#                     result.extend(candidates)
# #         print(len(result),'this much candidates')
# #         result_doc_idexes = [tupl[1] for tupl in result]
#         return result

    def binary_search(self,arr,query_distance,max_k):
        mid = len(arr)//2
        if arr[mid][0] > query_distance:
            arr = arr[:mid]
            if len(arr) <= max_k:
                return arr
            else:
                return self.binary_search(arr, query_distance, max_k)
        if arr[mid][0] < query_distance:
            arr = arr[mid:]
            if len(arr) <= max_k:
                return arr
            else:
                return self.binary_search(arr,query_distance,max_k)


In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from LSH import lsh as lsh_
# from LSH_TF import lsh as lsh_tf
import time
from sklearn.metrics.pairwise import cosine_similarity
# import nearpy

def load_mnist_data(num_samples,test_size=0.1):
    print('loading data')
    threshold = 70000
    if num_samples > threshold:
        num_iters = num_samples//threshold
        df_list = []
        for _ in range(num_iters):
            mnist = pd.read_csv('mnist-in-csv/mnist_train.csv',nrows=threshold)
            df_list.append((mnist))
        mnist = pd.concat(df_list)

    else:
        mnist = pd.read_csv('mnist-in-csv/mnist_train.csv', nrows=num_samples)
    mnist_data = np.array(mnist[mnist.columns[1:]])
    mnist_labels = np.array(mnist[mnist.columns[0]])
    X_train, X_test, y_train, y_test = split_data(x=mnist_data, y=mnist_labels, test_size=test_size)
    # X_train = X_train / 255
    # X_test = X_test / 255
    print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
    return X_train, X_test, y_train, y_test


def split_data(x, y, test_size):
    size = len(x)
    train_idx_stop = int(size * (1 - test_size))
    test_idx_start = train_idx_stop
    test_idx_stop = train_idx_stop + int(size * test_size)
    x_train = x[0:train_idx_stop]
    x_test = y[0:train_idx_stop]
    y_train = x[test_idx_start:test_idx_stop]
    y_test = y[test_idx_start:test_idx_stop]
    return x_train, x_test, y_train, y_test


def exhaustive_search(query_doc,candidates,labels):
    # print(len(candidates),'length of candidates')
    sims = cosine_similarity(candidates,query_doc)
    max_sim_idx = sims.argmax()
    return candidates[max_sim_idx], labels[max_sim_idx]


def candidate_label_distribution(candidates,y_train):
    candidate_label_dist = {i: 0 for i in range(10)}
    for doc_tuple in candidates:
        # candidates is list of tuple, each tuple contains distance from random projection vector and doc index
        candidate_label_dist[y_train[doc_tuple[1]]] += 1
    return candidate_label_dist


def check_accuracy(x_test, y_test,X_train,y_train, lsh):
    acc = 0
    for x,y in zip(x_test,y_test):
        candidate_docs,labels = lsh.fast_query(x)
        if len(candidate_docs) > 0:
            most_similar_doc,prediction = exhaustive_search(query_doc=x.reshape(1, -1),
                                                            candidates=candidate_docs, labels=labels)
            if prediction == y:
                acc += 1
    return acc/len(x_test)

def check_accuracy_tf(x_test, y_test,X_train,y_train, lsh):
    acc = 0
#     print('number of test docs',x_test.shape)
#     for x,y in zip(x_test,y_test):
        # print('>>>>',x,y)
    fast_query_candidates = lsh.fast_query(x_test)
#     print(len(fast_query_candidates), 'fast_query_candidates length')
#     print(fast_query_candidates[0], )
#     candidate_docs = X_train[fast_query_candidates]
    if len(candidate_docs) > 0:
        most_similar_arg_max = exhaustive_search(query_doc=x.reshape(1, -1), candidates=candidate_docs)
        most_similar_doc_idx = fast_query_candidates[most_similar_arg_max]
        prediction = y_train[most_similar_doc_idx]
        # print(prediction,y)
        if prediction == y:
            acc += 1
    return acc/len(x_test)


def main():
    X_train, y_train, X_test, y_test = load_mnist_data(num_samples=3000,test_size=0.1)
    lsh = lsh_(hash_size=12, data_dim=X_train.shape[1], num_tables=10)

    t1 = time.time()
    lsh.fit(X_train)
    print('total time taken for fitting', time.time() - t1)
    rand_doc_idx = 12
    query = X_test[rand_doc_idx]
    print('label of query doc ', y_test[rand_doc_idx])

    t1 = time.time()
    candidates = (lsh.query(query))
    print('total time taken for prediction', time.time() - t1)
    print('number of candidates:', len(candidates))
    print('candidate label distribution:', candidate_label_distribution(candidates,y_train))

    print('hash tables dist', lsh.hash_table_dist())
    t1 = time.time()
    fast_query_candidates = lsh.fast_query(query)
    print('total time taken for prediction using fast_query', time.time() - t1)
    candidate_label_dist = {i: 0 for i in range(10)}
    for doc_idx in fast_query_candidates:
        # candidates is list of tuple, each tuple contains distance from random projection vector and doc index
        candidate_label_dist[y_train[doc_idx]] += 1
    print('candidate label distribution:',candidate_label_dist )
    print('fast_query_candidates ',fast_query_candidates)
    print('fast_query_candidates length',len(fast_query_candidates))
    candidate_docs = X_train[fast_query_candidates]
    most_similar_arg_max = exhaustive_search(query_doc=query.reshape(1,-1), candidates=candidate_docs)
    most_similar_doc_idx = fast_query_candidates[most_similar_arg_max]
    print('label for the query is ', y_train[most_similar_doc_idx], 'idx', most_similar_doc_idx)


def main_2():
    x_train, y_train, x_test, y_test = load_mnist_data(num_samples=7000, test_size=0.1)
    lsh = lsh_(hash_size=8, data_dim=x_train.shape[1], num_tables=5)
    t1 = time.time()
    lsh.fit(x_train)
    print('total time taken for fitting', time.time() - t1)
    t1 = time.time()
    accuracy = check_accuracy(x_test, y_test,x_train,y_train,lsh)
    print('accuracy is ', accuracy)
    print('total time taken for checking accuracy for {} docs is'.format(len(x_test)), time.time() - t1)


def main_tf():
    x_train, y_train, x_test, y_test = load_mnist_data(num_samples=7000, test_size=0.1)
    lsh_ = lsh(hash_size=8, data_dim=x_train.shape[1], num_tables=5,random_type=None)
    t1 = time.time()
    lsh_.fit(x_train,y_train)
    print('total time taken for fitting using tensorflow', time.time() - t1)
    t1 = time.time()
    accuracy = check_accuracy(x_test, y_test,x_train,y_train,lsh_)
    print('accuracy is ', accuracy)
    print('total time taken for checking accuracy for {} docs is'.format(len(x_test)), time.time() - t1)
#     print(lsh_.hash_table_dist())

# main_2()
# main_tf()
#

# tf.test.is_gpu_available()
# tf.test.gpu_device_name()




#
# from nearpy import Engine
# from nearpy.hashes import RandomBinaryProjections
#
# # Dimension of our vector space
# dimension = 784
#
# # Create a random binary hash with 10 bits
# rbp = RandomBinaryProjections('rbp', 10)
#
# # Create engine with pipeline configuration
# engine = Engine(dimension, lshashes=[rbp])
#
# # Index 1000000 random vectors (set their data to a unique string)
# for img_vec in X_train:
#     engine.store_vector(img_vec, 'data_%d' % index)
#
# # Create random query vector
#
# query = X_test[1]
# # Get nearest neighbours
# t1 = time.time()
# N = engine.neighbours(query)
# # check_accuracy(X_test, y_test,X_train,y_train)
# print('time taken for query', time.time()-t1)
# show_mnist_image(query)

In [93]:
main_tf()

loading data
(6300, 784) (6300,) (700, 784) (700,)
TENSORFLOW GPU $$$$$$$$$$$$$ available GPU is -->>, 
fitting
is GPU availabale -> False
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

<class 'numpy.ndarray'> type(distance_matrix) (6300, 8)
total keys 6300
unique_keys 163
time taken for matmul  0.1644916534423828
[4362 2945 2445 ... 1420  554 2796] 6300
<class 'numpy.ndarray'> type(distance_matrix) (6300, 8)
total keys 6300
unique_keys 170
time taken for matmul  0.04644298553466797
[4522 1412 1123 ... 1271 1618 5972] 6300
<class 'numpy.ndarray'> type(distance_matrix) (6300, 8)
total keys 6300
unique_keys 156
time taken for matmul  0.03956294059753418
[4552 2786 1093 ... 6039 5965 6063] 6300
<class 'numpy.ndarray'> type(distance_matrix) (6300, 8)
total keys 6300
unique_keys 238
time taken for matmul  0.054404497146606445
[3475 5659 3150 ... 6013  944 6139] 6300
<class 'numpy.ndarray'> type(distance_matrix) (6300, 8)
total keys 6300
unique_ke

In [ ]:
assert data.shape[1] == dim, 'dimension of input data is {} and dimension in LSH object is {}'.format(
    data.shape, self.dim)
# sess = tf.Session()
# sess = tf.compat.v1.InteractiveSession()
print('fitting')
gpu_availability = tf.test.is_gpu_available()
print('is GPU availabale ->',gpu_availability)
#         if gpu_availability:
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
#         with tf.device("/GPU:0"):
tf.compat.v1.disable_eager_execution()
#         print('creating session on gpu, eager execution disabled ')
data_ph = tf.compat.v1.placeholder("float", [None, self.dim])
rand_hash_vec_ph = tf.compat.v1.placeholder("float", [None, self.dim])
distance_matrix_ = tf.matmul(data_ph, tf.transpose(rand_hash_vec_ph))
euclidean_dist_ = tf.sqrt(tf.reduce_sum(distance_matrix_ ** 2, axis=1))
# data_ph = tf.convert_to_tensor(data,dtype=tf.float32)

distance_matrix,euclidean_dist = session.run([distance_matrix_, euclidean_dist_],
                                               feed_dict={data_ph: data,
                                                          rand_hash_vec_ph: rand_vec})
keys = list(map(self.make_hash_key, (distance_matrix > 0).astype('int').astype('str')))


# for rand_vec, hash_table in zip(self.random_vectors, self.hash_tables):
#     # rand_hash_vec_ph = rand_vec
#     # distance_matrix_ = tf.matmul(data_ph, tf.transpose(rand_hash_vec_ph))
#     # euclidean_dist_ = tf.sqrt(tf.reduce_sum(distance_matrix_ ** 2, axis=1))
#     t1 = time.time()
#     distance_matrix,euclidean_dist = session.run([distance_matrix_, euclidean_dist_],
#                                               feed_dict={data_ph: data,
#                                                          rand_hash_vec_ph: rand_vec})
#     print('time taken for matmul ', time.time() - t1)
#     # distance_matrix, euclidean_dist = np.array(distance_matrix_), np.array(euclidean_dist_)
#     keys = list(map(self.make_hash_key, (distance_matrix > 0).astype('int').astype('str')))
#     # print('euclidean dist', (distance_matrix > 0).astype('int'))

#     # the keys contain string of length=hash_size (2 bits or 3 bits..) for each document.
#     # Eg '00','01','10','11'  for hash_size of 2 bits.
#     unique_keys = set(keys)
#     for key in unique_keys:
#         hash_table[key] = []
#     assert len(keys) == len(euclidean_dist), 'shape of euclidean dist matrix is {} and length of keys list'\
#                                              ' is {}. They must match'.format(len(euclidean_dist), len(keys))
#     sorted_keys = [(dist,key,key_idx) for dist, key, key_idx in
#                    sorted(zip(list(euclidean_dist),keys,range(len(keys))),  key=lambda pair: pair[0])]
#     # key_idx represents the document index in original data. We need to preserve this info before sorting
#     # the points in one bucket based on their distances from randomly projected vectors.
#     for key in sorted_keys:  # (dist,key,key_idx)
#         hash_table[key[1]].append((key[0],key[2]))  # (distance, doc_idx) <<<<<<<< very important
#         # each key:value pair in hash table looks like this
#         # '01':[(euclidean_dist, document_idx_in_data), () ,() , ......]
#         # '01' is a bucket, in which a document might be present in.
# #         else:
# #             print('fitting skipped')
